In [1]:
#| label: libraries
#| include: false
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3

## Elevator pitch

_The code and analysis below looks to explore the KPIs and different information to better understand batting averages and player salaries. Leveraging Python Pandas and Altair, provided are interesting insights that will lead to better informed descision as to wich teams and players are performing better and undertanding historical trends. By seamlessly reading data from a SQLite database with Python Pandas, it ensures comprehensive and accurate performance analysis. Then, with captivating visualizations created using Python Altair, you can uncover trends, patterns, and correlations, empowering you to optimize training, make informed decisions, and gain that competitive edge._


In [2]:
#| label: project-data
#| code-summary: Read and format project data

# SQLLite File
sqlFile = 'lahmansbaseballdb.sqlite'
#Connection
conn = sqlite3.connect(sqlFile)

__Highlight the Questions and Tasks__

## BYU-I Baseball Players

__Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__

_It looks like playerID: “lindsma01” has had a much better salary over the years as compared to playerID: “stephga01”. A couple things to consider when looking at the results is the year that is being associated with each salary observation. The highest salaries tend to be in the most recent year so we need to adjest for inflation to really get an accurate salary comparison bewteen the players._


In [3]:
#| label: Q1
#| code-summary: Read and format data

# SQL Query
sqlQueryPlayers = f"""SELECT
                        s.playerID,
                        cp1.schoolID,
                        s.yearID,
                        s.teamID,
                        s.salary
                    FROM salaries s
                        JOIN (
                            SELECT
                                cp2.playerID,
                                cp2.schoolID
                            FROM collegeplaying cp2
                            WHERE cp2.schoolID IN (
                                SELECT
                                    s.schoolID
                                FROM schools s
                                WHERE s.name_full LIKE '%Brigham Young University-Idaho%'
                            )
                        ) cp1
                            ON cp1.playerID = s.playerID
                    GROUP BY 1,2,3
                    ORDER BY 5 DESC"""

# Save results to variable
data_players = pd.read_sql(sql=sqlQueryPlayers,con=conn)

# Print results
data_players

,playerID,schoolID,yearID,teamID,salary
0,lindsma01,idbyuid,2014,CHA,4000000.0
1,lindsma01,idbyuid,2012,BAL,3600000.0
2,lindsma01,idbyuid,2011,COL,2800000.0
3,lindsma01,idbyuid,2013,CHA,2300000.0
4,lindsma01,idbyuid,2010,HOU,1625000.0
5,stephga01,idbyuid,2001,SLN,1025000.0
6,stephga01,idbyuid,2002,SLN,900000.0
7,stephga01,idbyuid,2003,SLN,800000.0
8,stephga01,idbyuid,2000,SLN,550000.0
9,lindsma01,idbyuid,2009,FLO,410000.0


In [28]:
# SQL Query
testy = f"""SELECT
                *
            FROM batting a
            WHERE playerID = 'addybo01'
                AND yearID = 1871
            GROUP BY 1
            LIMIT 2"""

# Save results to variable
caca = pd.read_sql(sql=testy,con=conn)

# Print results
caca

,ID,playerID,yearID,stint,teamID,team_ID,lgID,G,G_batting,AB,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,2,addybo01,1871,1,RC1,7,NA,25,None,118,...,13,8,1,4,0,None,None,None,None,0


## Batting Averages

__This three-part question requires you to calculate batting average (number of hits divided by the number of at-bats):__

_The equation that I used to calculate the batting average is: (Hits + Doubles + (2 * Triples) + (3 * Homeruns)) / At bats_

__Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report.__

In [4]:
#| label: Q2_1
#| code-summary: Read and format data

# SQL Query
sqlQueryBattingAVG_1 = f"""SELECT
                            playerID,
                            yearID,
                            (b.H + b."2B" + (2 * b."3B") + (3 * b.HR) / b.AB) 'battingAVG'
                        FROM batting b
                        WHERE b.AB > 0
                        GROUP BY 1,2
                        ORDER BY 3 DESC, 1
                        LIMIT 5"""

# Save results to variable                        
data_batting_avg_1 = pd.read_sql(sql=sqlQueryBattingAVG_1,con=conn)

# Print results
data_batting_avg_1

,playerID,yearID,battingAVG
0,cobbty01,1911,343
1,sislege01,1920,342
2,manushe01,1928,328
3,kleinch01,1930,325
4,hornsro01,1922,324


__Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results.__

In [5]:
#| label: Q2_2
#| code-summary: Read and format data

# SQL Query
sqlQueryBattingAVG_2 = f"""SELECT
                                playerID,
                                yearID,
                                (b.H + b."2B" + (2 * b."3B") + (3 * b.HR) / b.AB) 'battingAVG'
                            FROM batting b
                            WHERE b.AB >= 10
                            GROUP BY 1,2
                            ORDER BY 3 DESC, 1
                            LIMIT 5"""

# Save results to variable                        
data_batting_avg_2 = pd.read_sql(sql=sqlQueryBattingAVG_2,con=conn)

# Print results
data_batting_avg_2

,playerID,yearID,battingAVG
0,cobbty01,1911,343
1,sislege01,1920,342
2,manushe01,1928,328
3,kleinch01,1930,325
4,hornsro01,1922,324


__Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results.__

In [6]:
#| label: Q2_3
#| code-summary: Read and format data

# SQL Query
sqlQueryBattingAVG_3 = f"""SELECT
                                playerID,
                                SUM(b.AB) 'Total At-Bats',
                                (b.H + b."2B" + (2 * b."3B") + (3 * b.HR) / b.AB) 'battingAVG'
                            FROM batting b
                            WHERE b.AB >= 100
                            GROUP BY 1
                            HAVING SUM(b.AB) >= 100
                            ORDER BY 3 DESC, 1
                            LIMIT 5"""

# Save results to variable                        
data_batting_avg_alltime = pd.read_sql(sql=sqlQueryBattingAVG_3,con=conn)

# Print results
data_batting_avg_alltime

,playerID,Total At-Bats,battingAVG
0,jacksjo01,4801,316
1,williji01,5485,302
2,suzukic01,9885,292
3,keelewi01,8424,290
4,alexada01,2434,288


## Team performance over time

__Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph in Altair to visualize the comparison. What do you learn?__

_I chose to compare Arizona and Atlanta from 2000 onward because I wanted to get most recent track record of batting AVG by team per year. It’s interesting to the trends and switch of performance between the two teams around the year 2010. Atlanta had a better batting average from 2000 to 2010 and then Arizona maintained a better batting average for the most part up untill 2019._


In [7]:
#| label: Q3
#| code-summary: Read and format data

# SQL Query
sqlQueryBattingAVG_3 = f"""SELECT
                                t.name,
                                t.yearID,
                                (t.H + t."2B" + (2 * t."3B") + (3 * t.HR) / t.AB) 'battingAVG'
                            FROM teams t
                            WHERE t.name IN ('Atlanta Braves','Arizona Diamondbacks')
                                AND t.yearID >= 2000
                            ORDER BY 1,2"""

# Save results to variable                        
data_teams = pd.read_sql(sql=sqlQueryBattingAVG_3,con=conn)

# Print results
data_teams

,name,yearID,battingAVG
0,Arizona Diamondbacks,2000,1836
1,Arizona Diamondbacks,2001,1848
2,Arizona Diamondbacks,2002,1836
3,Arizona Diamondbacks,2003,1864
4,Arizona Diamondbacks,2004,1772
5,Arizona Diamondbacks,2005,1764
6,Arizona Diamondbacks,2006,1913
7,Arizona Diamondbacks,2007,1716
8,Arizona Diamondbacks,2008,1767
9,Arizona Diamondbacks,2009,1805


_include figures in chunks and discuss your findings in the figure._


In [8]:
#| label: Q3-chart
#| fig-align: center

#graph
graph = px.line(data_frame=data_teams,x='yearID',y='battingAVG',color='name',
                title= 'Batting Averages',
                labels= {
                    'battingAVG': 'Batting Average',
                    'yearID': 'Year',
                    'name': 'Team'
                })

graph.show()